# 1. Install required packages

In [1]:
!pip install opencv-contrib-python numpy

# 2. Create folders

In [2]:
import os

os.makedirs("dataset/1_Mahasweta", exist_ok=True)
os.makedirs("trainer", exist_ok=True)

# 3. Collect face images

In [3]:
import cv2

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)
count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.2, 5)

    for (x, y, w, h) in faces:
        face = cv2.resize(gray[y:y+h, x:x+w], (200, 200))
        path = f"dataset/1_Mahasweta/{str(count).zfill(4)}.jpg"
        cv2.imwrite(path, face)
        count += 1

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(frame, f"Saved: {count}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

    cv2.imshow("Collecting Images - Press 'q' to quit", frame)
    if cv2.waitKey(1) & 0xFF == ord('q') or count >= 50:
        break

cap.release()
cv2.destroyAllWindows()

# 4. Train the recognizer

In [4]:
import numpy as np

recognizer = cv2.face.LBPHFaceRecognizer_create()
faces, ids = [], []

folder = "dataset/1_Mahasweta"
for filename in os.listdir(folder):
    path = os.path.join(folder, filename)
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        faces.append(img)
        ids.append(1)

recognizer.train(faces, np.array(ids))
recognizer.save("trainer/trainer.yml")
print("Training complete.")

Training complete.


# 5. Real-time recognition

In [5]:
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("trainer/trainer.yml")
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.2, 5)

    for (x, y, w, h) in faces:
        face = cv2.resize(gray[y:y+h, x:x+w], (200, 200))
        label, confidence = recognizer.predict(face)
        name = "Mahasweta" if label == 1 and confidence < 60 else "Unknown"

        cv2.rectangle(frame, (x, y), (x+w, y+h), (255,0,0), 2)
        cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 2)

    cv2.imshow("Face Recognition - Press 'q' to quit", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()